In [1]:
!nvidia-smi

Tue Jul 18 18:42:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!unzip '/content/drive/MyDrive/data.zip'

Streaming output truncated to the last 5000 lines.
  inflating: data/traindev/copy-moved/images/c_0981.tif  
  inflating: data/traindev/copy-moved/images/c_0982.tif  
  inflating: data/traindev/copy-moved/images/c_0983.tif  
  inflating: data/traindev/copy-moved/images/c_0984.jpg  
  inflating: data/traindev/copy-moved/images/c_0985.jpg  
  inflating: data/traindev/copy-moved/images/c_0986.tif  
  inflating: data/traindev/copy-moved/images/c_0987.tif  
  inflating: data/traindev/copy-moved/images/c_0988.jpg  
  inflating: data/traindev/copy-moved/images/c_0989.tif  
  inflating: data/traindev/copy-moved/images/c_0990.jpg  
  inflating: data/traindev/copy-moved/images/c_0991.tif  
  inflating: data/traindev/copy-moved/images/c_0992.tif  
  inflating: data/traindev/copy-moved/images/c_0993.jpg  
  inflating: data/traindev/copy-moved/images/c_0994.tif  
  inflating: data/traindev/copy-moved/images/c_0995.jpg  
  inflating: data/traindev/copy-moved/images/c_0996.tif  
  inflating: data/tra

In [3]:
#importing required libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
# initializing image data generators with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [22]:
#preprocessing test images
test_generator = test_data_gen.flow_from_directory(
    '/content/data/test',
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

Found 498 images belonging to 3 classes.


In [23]:
#preprocessing train images
train_generator = train_data_gen.flow_from_directory(
    '/content/data/train',
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

Found 4482 images belonging to 3 classes.


In [24]:
#creating model structure
#1rst layer
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (48,48,1)))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

In [25]:
#2nd layer
model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

In [26]:
#flattening
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation = 'softmax'))

In [27]:
#compiling
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001, decay = 1e-6), metrics = ['accuracy'])

In [48]:
model_info = model.fit(
    train_generator,
    steps_per_epoch=4482 // 64,
    validation_data=test_generator,
    validation_steps=498 // 64,
    epochs=10
)

Epoch 1/10
70/70 [==============================] - 15s 214ms/step - loss: 0.2940 - accuracy: 0.8794 - val_loss: 1.7508 - val_accuracy: 0.4554
Epoch 2/10
70/70 [==============================] - 14s 205ms/step - loss: 0.2941 - accuracy: 0.8837 - val_loss: 1.8410 - val_accuracy: 0.4576
Epoch 3/10
70/70 [==============================] - 14s 201ms/step - loss: 0.2813 - accuracy: 0.8884 - val_loss: 1.8388 - val_accuracy: 0.4576
Epoch 4/10
70/70 [==============================] - 14s 205ms/step - loss: 0.2762 - accuracy: 0.8943 - val_loss: 1.8268 - val_accuracy: 0.4554
Epoch 5/10
70/70 [==============================] - 14s 206ms/step - loss: 0.2755 - accuracy: 0.8900 - val_loss: 1.9592 - val_accuracy: 0.4621
Epoch 6/10
70/70 [==============================] - 14s 206ms/step - loss: 0.2818 - accuracy: 0.8852 - val_loss: 1.8665 - val_accuracy: 0.4554
Epoch 7/10
70/70 [==============================] - 15s 209ms/step - loss: 0.2791 - accuracy: 0.8895 - val_loss: 1.8893 - val_accuracy: 0.4754

In [49]:
#saving the model and weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
print("saved")

saved


In [50]:
# saving trained model weight in .h5 file
model.save_weights('model.h5')